In [14]:


import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define the image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

# Define the directories for the training and validation datasets
train_dirs = [
     r'dataset/data3a/training/01-minor',
     r'dataset/data3a/training/02-moderate',
     r'dataset/data3a/training/03-severe',
     r'dataset/data2a/training/00-front',
     r'dataset/data2a/training/01-rear',
     r'dataset/data2a/training/02-side',
     r'dataset/data1a/training/00-damage',
     r'dataset/data1a/training/01-whole'
]

val_dirs = [
     r'dataset/data3a/validation/01-minor',
     r'dataset/data3a/validation/02-moderate',
     r'dataset/data3a/validation/03-severe',
     r'dataset/data2a/validation/00-front',
     r'dataset/data2a/validation/01-rear',
     r'dataset/data1a/validation/00-damage',
     r'dataset/data1a/validation/01-whole'
]

# Define the data augmentation strategy for the training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define the data augmentation strategy for the validation data
val_datagen = ImageDataGenerator(
    rescale=1./255
)

# Define the generator for the training data
train_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
).flow_from_directory(
    train_dirs[0],
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

for i in range(1, len(train_dirs)):
    train_generator1 = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    ).flow_from_directory(
        train_dirs[i],
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=True
    )
    train_generator = keras.preprocessing.image.Iterator(train_generator.index_array+train_generator1.index_array, train_generator.labels, train_generator.image_data_generator, train_generator.batch_size, train_generator.target_size, train_generator.data_format, train_generator.classes, train_generator.class_mode, train_generator.shuffle, train_generator.seed, train_generator.save_to_dir, train_generator.save_prefix, train_generator.save_format, train_generator.interpolation, train_generator.dtype)

# Define the generator for the validation data
val_generator = ImageDataGenerator(
    rescale=1./255
).flow_from_directory(
    val_dirs[0],
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

for i in range(1, len(val_dirs)):
    val_generator1 = ImageDataGenerator(
        rescale=1./255
    ).flow_from_directory(
        val_dirs[i],
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    val_generator = keras.preprocessing.image.Iterator(val_generator.index_array+val_generator1.index_array, val_generator.labels, val_generator.image_data_generator, val_generator.batch_size, val_generator.target_size, val_generator.data_format, val_generator.classes, val_generator.class_mode, val_generator.shuffle, val_generator.seed, val_generator.save_to_dir, val_generator.save_prefix, val_generator.save_format, val_generator.interpolation, val_generator.dtype)


# Define the model architecture
model = keras.models.Sequential([
    keras.layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=[IMG_SIZE, IMG_SIZE, 3]),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(8, activation='softmax')
])

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [ ]:
# Save the model
model.save("vehicle_damage_detection_model-try2.h5")

# Load the model
model = keras.models.load_model("vehicle_damage_detection_model-try2.h5")

# Define a function to preprocess the image
def preprocess_image(test-image.jpg):
    img = keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Define a function to make predictions on an image
def predict_image(image_path):
    # Preprocess the image
    img_array = preprocess_image(image_path)
    # Make a prediction
    predictions = model.predict(img_array)
    # Get the predicted class
    predicted_class = np.argmax(predictions[0])
    # Map the predicted class to the corresponding label
    if predicted_class == 0:
        damage = "not damaged"
        severity = "N/A"
        area = "N/A"
    elif predicted_class == 1:
        damage = "minor damage"
        severity = "minor"
        area = "front"
    elif predicted_class == 2:
        damage = "moderate damage"
        severity = "moderate"
        area = "rear"
    elif predicted_class == 3:
        damage = "moderate damage"
        severity = "moderate"
        area = "side"
    elif predicted_class == 4:
        damage = "severe damage"
        severity = "severe"
        area = "front"
    elif predicted_class == 5:
        damage = "severe damage"
        severity = "severe"
        area = "rear"
    elif predicted_class == 6:
        damage = "severe damage"
        severity = "severe"
        area = "side"
    elif predicted_class == 7:
        damage = "whole damage"
        severity = "N/A"
        area = "N/A"
    # Return the predicted result
    return {
        "damage": damage,
        "severity": severity,
        "area": area
    }

# Test the model using an image
image_path = "test_image.jpg"
result = predict_image(image_path)
print(result)
